# for Advent Calender 2019 Julia

### 目的地
1. Juliaを使って記号処理をしてみる。
2. Termのunificationまでを目指すが、代入あたりで終了するかもしれない。量が多くなるので。
3. 基本は手抜きで、手間のかかることは極力しない。
4. キーワード: Meta.parse, Expr, substitution, unification

### 定義
1. 項(Term)

- Term ::= Fterm::Expr | Variable | Constant
- Constant ::= Symbol{not in vars}
- Variable ::= Symbol{in vars}
- Vars :: List(Symbol)

Varsの要素のSymbolは変数とみなす。

実用的には、ConstatantにNumber、StringやCharを含めることもしたいが、ここでは説明/コードが長くなるので略する。

ExprはJuliaのMeta.parse()の出力。

2. 代入表現
まず、代入のメタな表現を定義する。

代入は{x/f(a),y/a}でxにf(y),yにaの代入を表したりするが、ここでは{(x,y)←(f(a),a)}のように書くことにする。

←の左側をV(σ)、右側をE(σ)と書く。つまり

σ    = {(x,y)←(f(a),a)}
V(σ) = (x,y)
E(σ) = (f(a),a)

である。

Juliaでの表現は、代入操作を定義するときに決める。


σを代入表現で、TをTermだとすると、Tにσを適用する代入操作は

subst(ν, T, σ)

と表記する。νはTに出現する変数のリストで、Tに出現しない変数を含んでいてもよい。
Juliaでは、νの型はVList = Array{Symbol,1}とする。

例

T = g(x,h(y))
とし、σを先の例とすると、この代入操作とその結果は次のようになる。

subst((x,y), g(x,h(y)), (f(a),a)) = g(f(a), h(a))

このように、この記事では、代入表現は単なるTermのリストとして表記する。
σの型は、TList = Array{Term, 1}とする。

V(σ)は、substの引数として与えるので、代入表現自体には変数リストは持たない。

4. Unification
T,S をTermとし、νをTとSに出現する変数のリストとするとき、TとSのunificationは

unify(ν, T, S)

と表記する。


Exprの例

In [1]:
e=Meta.parse("f(x,y)")
dump(e)

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol f
    2: Symbol x
    3: Symbol y


このように、Exprは(head,args)である。headは関数呼び出しなら:callで、他にもいくつかあるが、この記事では:callしかでてこない。特徴的なことは関数記号もargsの先頭要素として含まれるという点で、これは、ExprがLISPのS式の拡張になっているということのように見える。
他には、

In [2]:
e2=Meta.parse("f(g(x),a)")
e2

:(f(g(x), a))

### Julia 定義

In [3]:
FTerm=Expr
Term=Union{Symbol, Expr}

VList = Array{Symbol, 1}
TList = Array{Term, 1}


Array{Union{Expr, Symbol},1}

### Testを使う

In [4]:
using Test

一回目はtestは失敗し、定義したあともう一度実行すると成功する

### 基本関数

In [5]:
isvar(ν::VList, e::Symbol) = e in ν 

isvar (generic function with 1 method)

In [6]:
isvar(ν::VList, e::Expr) = false

isvar (generic function with 2 methods)

In [7]:
@testset "isvar(symbol)" begin
    @test isvar([:x,:y], :a) == false
    @test isvar([:x,:y], :x) == true
end

Test Summary: | Pass  Total
isvar(symbol) |    2      2


Test.DefaultTestSet("isvar(symbol)", Any[], 2, false)

In [8]:
@testset "isvar(expr)" begin
    @test isvar([:x,:y], :(f(x,y))) == false
    @test isvar([:x,:y], :(f(g(x),y))) == false
end

Test Summary: | Pass  Total
isvar(expr)   |    2      2


Test.DefaultTestSet("isvar(expr)", Any[], 2, false)

### 代入操作

substの第三引数をTListにすると、subst0の第三引数がUnionになって型が一致しないエラーになったのでArrayにした。

In [9]:
function subst(ν::VList, t::Symbol, σ::Array)
    map(function(ν, σ) t = subst0(ν, t, σ) end, ν, σ)
    t
end

function subst0(ν::Symbol, t::Symbol, σ::Term)
    if ν == t; return σ
        else return t
    end
end

function subst(ν::VList, t::Expr, σ::Array)
    map(function(ν, σ) t = subst0(ν, t, σ) end, ν, σ)
    t
end

function subst0(ν::Symbol, t::Expr, σ::Term)
    nargs = []
    for e in t.args
        push!(nargs, subst0(ν, e, σ))
    end
    t.args=nargs
    return t
end

subst0 (generic function with 2 methods)

function subst(ν::VList, t::Symbol, σ::TList)
    map(function(ν, σ) t = subst0(ν, t, σ) end, ν, σ)
    t
end

function subst0(ν::Symbol, t::Symbol, σ::Term)
    if ν == t; return σ
        else return t
    end
end

function subst(ν::VList, t::Expr, σ::TList)
    map(function(ν, σ) t = subst0(ν, t, σ) end, ν, σ)
    t
end

function subst0(ν::Symbol, t::Expr, σ::Term)
    nargs = []
    for e in t.args
        push!(nargs, subst0(ν, e, σ))
    end
    t.args=nargs
    return t
end

In [10]:
@testset "subst0 on symbol" begin
    @test subst0(:a, :a, :a) == :a
    @test subst0(:b, :a, :a) == :a
    @test subst0(:a, :a, :b) == :b
    @test subst0(:a, :a, :(f(x))) == :(f(x))
end

Test Summary:    | Pass  Total
subst0 on symbol |    4      4


Test.DefaultTestSet("subst0 on symbol", Any[], 4, false)

In [11]:
@testset "subst on symbol" begin
    @test subst([:a], :a, [:a]) == :a
    @test subst([:b], :a, [:a]) == :a
    @test subst([:a], :a, [:b]) == :b
    @test subst([:a], :a, [:(f(x))]) == :(f(x))
end

Test Summary:   | Pass  Total
subst on symbol |    4      4


Test.DefaultTestSet("subst on symbol", Any[], 4, false)

In [12]:
@testset "subst0 on Expr" begin
    @test subst0(:a, :(f(x)), :a) == :(f(x))
    @test subst0(:a, :(f(x)), :b) == :(f(x))
    @test subst0(:x, :(f(x)), :a) == :(f(a))
    @test subst0(:y, :(f(x)), :(f(b))) == :(f(x))
    # the next is ng when this line was deleted???
    @test subst0(:x, :(f(x)), :(g(b))) == :(f(g(b)))

end

Test Summary:  | Pass  Total
subst0 on Expr |    5      5


Test.DefaultTestSet("subst0 on Expr", Any[], 5, false)

In [13]:
@testset "subst on Expr" begin
    @test subst([:a,:b], :(f(x)), [:a,:b]) == :(f(x))
    @test subst([:a,:b], :(f(x)), [:b,:c]) == :(f(x))
    @test subst([:x,:b], :(f(x)), [:a,:b]) == :(f(a))
    @test subst([:a,:x], :(f(x)), [:a,:b]) == :(f(b))
    @test subst([:x,:y], :(f(x,y)), [:d,:e]) == :(f(d,e))
    @test subst([:x,:y], :(f(x,y,z)), [:(g(b)),:y]) == :(f(g(b),y,z))
    @test subst([:x,:y], :(f(x,y,z)), [:(g(b)),:(h(d))]) == :(f(g(b),h(d),z))
    @test subst([:x,:y], :(f(x,z,y)), [:(g(b)),:(h(d))]) == :(f(g(b),z,h(d)))
end

Test Summary: | Pass  Total
subst on Expr |    8      8


Test.DefaultTestSet("subst on Expr", Any[], 8, false)

### Unification

あれ? σ作るのたいへんではないか??


In [14]:
abstract type UnifyException <: Exception end
struct UnifyFAIL <: UnifyException
    left
    right
end
struct UnifyLOOP <: UnifyException 
    left
    right
end

案1 
σの中身を変えるとき、毎回、対応する変数の位置をしらべて、書き換えなくてはならない。
これはめんどうなので
一時的にDict{V}::Termに格納して、最後にTermだけ抜き出すとか・・・
案2
cheaplogicでは、インデックス計算している。
そんなに遅くはないような気もするが気持ち悪い。

★いつのまにか間違ったことをしていたと思う

たぶん、1つの式の中の変数の数はそれほど多くないので、インデックス計算でもそれほど性能悪化していないのだろう。
これをDictでやると、よけいに時間がかかりそう。メモリは余分に必要になる。

cheaplogicのままでよさそう。


In [ ]:
function vindex(vars::Vlist, v::Symbol)
 ix = findfirst(x->x==v,vars)
 if ix == nothing; return 0 end
 return ix
end

function putsigma(ν::VList, σ::TList, ix::Number, t::Term)
   σ[ix] = t 
end

In [ ]:
@testset "vindex" begin
    @test vindex([:x,:y,:z], :x) == 1
    @test vindex([:x,:y,:z], :y) == 2
    @test vindex([:x,:y,:z], :z) == 3
    @test vindex([:x,:y,:z], :w) == 0
end

@testset "putsigma" begin
    @test putsigma([:x,:y],[:x,:(f(y)),:y)],1,:a) == [:a,:(f(y))]
    @test_throws UnifyFAIL putsigma([:x,:y],[:x,:(f(y)),:y)],2,:a)
    
end


In [17]:
function unify(ν::VList, t1::Expr, t2::Expr)::TList
end

function unify(ν::VList, t1::Symbol, t2::Expr)::TList
    throw(UnifyFAIL(t1,t2))
end

function unify(ν::VList, t1::Expr, t2::Symbol)::TList
    throw(UnifyFAIL(t1,t2))
end

function unify(ν::VList, t1::Symbol, t2::Symbol)::TList
    if !isvar(ν, t1) && !isvar(ν, t2) && t1!=t2
    throw(UnifyFAIL(t1,t2))
    end
end

function unify1(ν::VList, t1::Array, t2::Array)::TList
end


unify1 (generic function with 1 method)

In [ ]:
@testset "unify normal" begin
    @test unify([], :x, :x) == []
    @test unify([:x], :x, :x) == [:x]
    @test unify([:x], :x, :a) == [:a]
    @test unify([:x,:y], :x, :a) == [:a,:y]
    
    @test unify([:x], :(f(x,y)), :(f(a,b))) = [:a,:b]

    @test unify([:x,:y], :x, :y) == [:y,:y] # an implementation choice
    @test unify([:x,:y], :y, :x) == [:x,:x] # an implementation choice
    
    @test unify([:x,:y],:(f(x,y)),:(f(y,x))) == [:y,:y] # an implementation choice

    @test unify([:x,:y,:z,:w], :(f(g(x),y,k(y))), :(f(z,h(z),w))) == [:x,:(h(g(x))),:(g(x)),:(k(h(g(x))))]
                
end

In [ ]:
@testset "unify fail" begin
    @test_throws UnifyFAIL unify([], :a, :b)
    @test_throws UnifyFAIL unify([x,y], :(f(x)), :(g(x)))
    @test_throws UnifyFAIL unify([x,y], :(f(a)), :(f(b)))
    @test_throws UnifyFAIL unify([x,y], :(f(g(x))), :(f(h(y))))
    
end


In [ ]:
@testset "unify loop error" begin
    @test_throws UnifyLOOP unify([:x], :x, :(f(x)))
    
    @test_throws UnifyLOOP unify([:x,:y], :(f(x,x)), :(f(x,g(y))))
    @test_throws UnifyLOOP unify([:x,:y], :(f(x,g(y))), :(f(x,x)))

    
end